<a href="https://colab.research.google.com/github/yenajo98/Amsterdam_housing_occupants/blob/main/Model_Testing_OtherEuropeancities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from termcolor import colored as cl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, ElasticNet
from sklearn.metrics import explained_variance_score as evs, r2_score as r2
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import skew, stats, pearsonr, norm
from collections import Counter
from sklearn.linear_model import LassoCV, LassoLarsCV, ElasticNetCV
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler

warnings.filterwarnings('ignore')
sns.set(style='white', context='notebook', palette='deep')
%config InlineBackend.figure_format = 'retina'  # set 'png' here when working on notebook
%matplotlib inline


In [21]:
import pandas as pd
weekdays=pd.read_csv('/content/amsterdam_weekdays.csv')
weekdays.head()

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508


In [22]:
new_dataset=pd.read_csv('/content/berlin_weekdays.csv')
new_dataset.head()

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,185.799757,Private room,False,True,2.0,True,0,0,10.0,98.0,1,3.582211,0.174706,105.063708,16.019042,148.941114,30.710638,13.42344,52.49150
1,1,194.914462,Private room,False,True,5.0,False,0,1,9.0,86.0,1,3.525410,0.511922,75.339529,11.487002,106.442356,21.947685,13.46800,52.51900
2,2,176.217631,Private room,False,True,2.0,False,0,0,9.0,91.0,1,3.801713,0.281397,73.669176,11.232324,105.440205,21.741048,13.47096,52.51527
3,3,207.768533,Private room,False,True,3.0,True,0,0,10.0,97.0,1,0.982408,0.705573,133.187409,20.307057,198.233362,40.874362,13.42281,52.53139
4,4,150.743199,Private room,False,True,2.0,False,0,0,10.0,99.0,1,8.869697,2.187188,39.860151,6.077469,50.996308,10.515090,13.52440,52.47842


## DATA PREPROCESSING

In [23]:
#Drop the unnamed column
weekends = weekends.drop(weekends.columns[0], axis=1)
weekdays = weekdays.drop(weekdays.columns[0], axis=1)
# Drop the specified columns
columns_to_drop = ['attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm']
weekends = weekends.drop(columns_to_drop, axis=1)
weekdays = weekdays.drop(columns_to_drop, axis=1)
weekends = weekends.rename(columns={'realSum': 'Price'})
weekdays = weekdays.rename(columns={'realSum': 'Price'})
# Filter out boolean format variables
bool_cols = weekdays.select_dtypes(include=bool).columns

# Convert boolean variables to object variable type
weekdays[bool_cols] = weekdays[bool_cols].astype(str)


# Filter out boolean format variables
bool_cols = weekends.select_dtypes(include=bool).columns

# Convert boolean variables to object variable type
weekends[bool_cols] = weekends[bool_cols].astype(str)
# One-hot encode categorical features
weekdays = pd.get_dummies(weekdays)
weekdays

,Price,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,lng,lat,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,room_shared_False,room_shared_True,room_private_False,room_private_True,host_is_superhost_False,host_is_superhost_True
0,194.033698,2.0,1,0,10.0,93.0,1,5.022964,2.539380,4.90569,52.41772,0,1,0,1,0,0,1,1,0
1,344.245776,4.0,0,0,8.0,85.0,1,0.488389,0.239404,4.90005,52.37432,0,1,0,1,0,0,1,1,0
2,264.101422,2.0,0,1,9.0,87.0,1,5.748312,3.651621,4.97512,52.36103,0,1,0,1,0,0,1,1,0
3,433.529398,4.0,0,1,9.0,90.0,2,0.384862,0.439876,4.89417,52.37663,0,1,0,1,0,0,1,1,0
4,485.552926,2.0,0,0,10.0,98.0,1,0.544738,0.318693,4.90051,52.37508,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2486.115342,2.0,0,0,10.0,100.0,1,2.375833,1.436054,4.85869,52.37677,1,0,0,1,0,1,0,1,0
1099,233.637194,2.0,1,0,10.0,90.0,1,4.719736,0.322263,4.83611,52.34910,0,1,0,1,0,0,1,1,0
1100,317.062311,2.0,1,0,10.0,92.0,1,0.598355,0.751993,4.88897,52.37798,0,1,0,1,0,0,1,1,0
1101,1812.855904,4.0,0,0,8.0,84.0,5,1.943020,0.388532,4.90688,52.35794,1,0,0,1,0,1,0,1,0


In [24]:
#Drop the unnamed column
new_dataset = new_dataset.drop(new_dataset.columns[0], axis=1)
# Drop the specified columns
columns_to_drop = ['attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm']
new_dataset = new_dataset.drop(columns_to_drop, axis=1)

new_dataset = new_dataset.rename(columns={'realSum': 'Price'})

# Filter out boolean format variables
bool_cols = new_dataset.select_dtypes(include=bool).columns

# Convert boolean variables to object variable type
new_dataset[bool_cols] = new_dataset[bool_cols].astype(str)


# Filter out boolean format variables
bool_cols = new_dataset.select_dtypes(include=bool).columns

# Convert boolean variables to object variable type
new_dataset[bool_cols] = new_dataset[bool_cols].astype(str)
# One-hot encode categorical features
new_dataset = pd.get_dummies(new_dataset)
new_dataset

,Price,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,lng,lat,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,room_shared_False,room_shared_True,room_private_False,room_private_True,host_is_superhost_False,host_is_superhost_True
0,185.799757,2.0,0,0,10.0,98.0,1,3.582211,0.174706,13.42344,52.49150,0,1,0,1,0,0,1,0,1
1,194.914462,5.0,0,1,9.0,86.0,1,3.525410,0.511922,13.46800,52.51900,0,1,0,1,0,0,1,1,0
2,176.217631,2.0,0,0,9.0,91.0,1,3.801713,0.281397,13.47096,52.51527,0,1,0,1,0,0,1,1,0
3,207.768533,3.0,0,0,10.0,97.0,1,0.982408,0.705573,13.42281,52.53139,0,1,0,1,0,0,1,0,1
4,150.743199,2.0,0,0,10.0,99.0,1,8.869697,2.187188,13.52440,52.47842,0,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,232.074413,2.0,1,0,10.0,96.0,1,3.482221,0.509602,13.45614,52.50351,0,1,0,1,0,0,1,1,0
1280,175.049079,2.0,0,0,10.0,100.0,1,11.002848,2.377031,13.50477,52.44047,1,0,0,1,0,1,0,1,0
1281,158.689352,2.0,0,0,9.0,91.0,1,11.313156,2.531878,13.51031,52.43937,1,0,0,1,0,1,0,1,0
1282,134.617182,4.0,1,0,10.0,93.0,1,15.166191,2.754843,13.53301,52.40712,0,1,0,1,0,0,1,1,0


## Modelling

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

# Split your data into X and y
X = weekdays.drop("Price", axis=1)
y = weekdays["Price"]

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the model
rf = RandomForestRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Initialize the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model using the best parameters
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(X_train, y_train)

# Now you can use best_rf to make predictions
y_pred_train = best_rf.predict(X_train)
y_pred_test = best_rf.predict(X_test)

# Calculate metrics on the training set
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# Calculate metrics on the testing set
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

# Print the metrics
print(f'Training MSE: {mse_train}')
print(f'Training RMSE: {rmse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training R2: {r2_train}')
print()
print(f'Testing MSE: {mse_test}')
print(f'Testing RMSE: {rmse_test}')
print(f'Testing MAE: {mae_test}')
print(f'Testing R2: {r2_test}')
print(f'Best parameters: {best_params}')

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Training MSE: 1.039785264863088e-07
Training RMSE: 0.00032245701494355616
Training MAE: 0.00023748820200944528
Training R2: 0.9144343395329714

Testing MSE: 3.1047410357543756e-07
Testing RMSE: 0.0005572020312018232
Testing MAE: 0.0004300611669026167
Testing R2: 0.7086690943843066
Best parameters: {'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


#### Testing the other city (Berlin)

In [26]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

# Prepare the new dataset
X_new = new_dataset.drop("Price", axis=1)
y_new = new_dataset["Price"]

# Split the new dataset into training and testing sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# Create an instance of the model
rf = RandomForestRegressor()

# Load the best parameters
best_params = {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto'}
best_rf = RandomForestRegressor(**best_params)

# Train the model using the best parameters
best_rf.fit(X_train_new, y_train_new)

# Make predictions on the new dataset
y_pred_train_new = best_rf.predict(X_train_new)
y_pred_test_new = best_rf.predict(X_test_new)

# Calculate metrics on the training set of the new dataset
mse_train_new = mean_squared_error(y_train_new, y_pred_train_new)
rmse_train_new = sqrt(mse_train_new)
mae_train_new = mean_absolute_error(y_train_new, y_pred_train_new)
r2_train_new = r2_score(y_train_new, y_pred_train_new)

# Calculate metrics on the testing set of the new dataset
mse_test_new = mean_squared_error(y_test_new, y_pred_test_new)
rmse_test_new = sqrt(mse_test_new)
mae_test_new = mean_absolute_error(y_test_new, y_pred_test_new)
r2_test_new = r2_score(y_test_new, y_pred_test_new)

# Print the metrics for the new dataset
print(f'Training MSE (New): {mse_train_new}')
print(f'Training RMSE (New): {rmse_train_new}')
print(f'Training MAE (New): {mae_train_new}')
print(f'Training R2 (New): {r2_train_new}')
print()
print(f'Testing MSE (New): {mse_test_new}')
print(f'Testing RMSE (New): {rmse_test_new}')
print(f'Testing MAE (New): {mae_test_new}')
print(f'Testing R2 (New): {r2_test_new}')


Training MSE (New): 8891.639892050558
Training RMSE (New): 94.29549242700077
Training MAE (New): 39.39481412016742
Training R2 (New): 0.8542938038921881

Testing MSE (New): 13209.95392897301
Testing RMSE (New): 114.93456368287569
Testing MAE (New): 62.60822845249552
Testing R2 (New): 0.35676118292123715
